In [4]:
%%capture cap --no-stderr

import shutil
import sys
import json
import datetime
import os, glob, h5py
import numpy as np
import matplotlib.pyplot as plt
from datafed.CommandLib import API

from PyQt5.QtGui import QFont 
from PyQt5.QtCore import * 
from PyQt5.QtWidgets import * 
sys.path.append('../')
from PLDForm import GenerateForm

if __name__ == "__main__":
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    
    custom_font = QFont("Times", 10)

    app.setFont(custom_font, "QGroupBox")
    app.setFont(custom_font, "QComboBox")
    app.setFont(custom_font, "QLabel")
    app.setFont(custom_font, "QLineEdit")
    app.setFont(custom_font, "QPlainTextEdit")

    window = GenerateForm(version="plume")
    window.show()
    app.exec_()

In [2]:
# save outputs to log file
log_file ='../../2.logs/'+datetime.datetime.today().strftime("%m-%d-%Y")+'.txt'
with open(log_file, 'a') as f:
    f.write(cap.stdout)